# Modelisation supervisee

## A. Pre-requis et Nettoyage

In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [25]:
df = pd.read_csv("dataset/dataset-clean-v2.csv")

In [26]:
df.head()

,Unnamed: 0,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,preproc_tags,preproc_body,preproc_title
0,0,Is gettimeofday() guaranteed to be of microsec...,"<p>I am porting a game, that was originally wr...",<linux><winapi><visual-c++><unix><timer>,88,107,45385,10,"['linux', 'winapi', 'visual-c++', 'unix', 'tim...","['port', 'game', 'originally', 'write', 'win32...","['gettimeofday', 'guarantee', 'microsecond', '..."
1,1,Decoding T-SQL CAST in C#/VB.NET,<p>Recently our site has been deluged with the...,<c#><sql><vb.net><ascii><hex>,109,68,6252,2,"['c#', 'sql', 'vb.net', 'ascii', 'hex']","['recently', 'site', 'deluge', 'resurgence', '...","['decode', 'sql', 'cast']"
2,2,Displaying Flash content in a C# WinForms appl...,<p>What is the best way to display <code>Flash...,<c#><winforms><flash><adobe><macromedia>,1037,40,37517,2,"['c#', 'winforms', 'flash', 'adobe', 'macromed...","['good', 'way', 'display', 'flash', 'content',...","['display', 'flash', 'content', 'winforms', 'a..."
3,3,Why doesn't SQL Full Text Indexing return resu...,"<p>For instance, my query is like the followin...",<sql><sql-server><sql-server-2005><indexing><f...,1042,26,9479,2,"['sql', 'sql-server', 'sql-server-2005', 'inde...","['instance', 'query', 'like', 'follow', 'sql',...","['sql', 'text', 'indexing', 'return', 'result'..."
4,4,How can I unit test Flex applications from wit...,<p>I'm currently working on an application wit...,<apache-flex><eclipse><unit-testing><build-aut...,2222,18,2764,4,"['apache-flex', 'eclipse', 'unit-testing', 'bu...","['currently', 'work', 'application', 'frontend...","['unit', 'test', 'flex', 'application', 'ide',..."


In [27]:
def convert_columns(text):
    """
    Convertit une chaîne de caractères en liste de mots en remplaçant certains caractères spéciaux et en séparant les éléments.

    Args:
        text (str): La chaîne de caractères à traiter. Celle-ci peut contenir des crochets, des virgules et des guillemets.

    Returns:
        list: Une liste de mots extraits de la chaîne, avec les caractères spéciaux remplacés par des espaces.
    """
    return text.replace("[", " ").replace("]", " ").replace(",", " ").replace("'", " ").split()

In [28]:
# Appliquer la fonction 'convert_columns' aux variables 'preproc_tags', 'preproc_body', 'preproc_title'

df['preproc_title'] = df['preproc_title'].apply(convert_columns)
df['preproc_body'] = df['preproc_body'].apply(convert_columns)
df['preproc_tags'] = df['preproc_tags'].apply(convert_columns)

In [29]:
# Fonction pour le pre-processing du texte

# Charger le modèle de langue
nlp = spacy.load("en_core_web_sm")

# Liste des mots spécifiques à conserver (comme "C++", "C#", etc.)
whitelist = ["c#", "c++"]

# Fonction pour le traitement du texte
def preprocess_text(text):
    # Enlever les balises HTML
    #soup = BeautifulSoup(text, "lxml").get_text()
    doc = nlp(text)  # Traiter le texte avec spaCy
    
    # Expression régulière pour vérifier les caractères anglais uniquement (ASCII)
    def is_english(token):
        return re.match(r'^[a-zA-Z0-9+.#]+$', token)

    tokens = [
        # Utiliser le lemme sauf si le mot est dans la whitelist
        token.lemma_.lower() if token.lemma_.lower() not in whitelist else token.text.lower()
        for token in doc 
        if not token.is_stop                         # Ne pas inclure les stopwords
        and not token.is_punct                       # Ne pas inclure la ponctuation
        and not token.like_num                       # Ne pas inclure les chiffres
        and len(token.lemma_) >= 3                    # Exclure les tokens trop courts
        and (is_english(token.text) or token.text.lower() in whitelist)  # Garder les mots anglais ou ceux de la whitelist
    ]
    print(tokens)
    return tokens

## B. Features Engineering

In [30]:
# Creation de la variable 'text' qui regroupe les colonnes 'preproc_title' et 'preproc_body'

df['corpus'] = df['preproc_body'].apply(lambda x: ' '.join(x)) + ' ' + df['preproc_title'].apply(lambda x: ' '.join(x))

In [31]:
# Creation de la variable 'tags' qui regroupe les colonnes 'preproc_tags'

df['tags'] = df['preproc_tags'].apply(lambda x: ' '.join(x))

## C. Modelisation

### 1 : Regression Logistique avec Bag of Words

#### Creation du Bag-Of-Words

In [33]:
# Initialiser le CountVectorizer
vectorizer = CountVectorizer()

# Appliquer le vectorizer sur le corpus
X_bow = vectorizer.fit_transform(df['corpus'])

# Afficher la forme de la matrice BoW
print(X_bow.shape)

(50000, 118148)


#### Separation des donnees en ensemble d'entrainement et de test

In [34]:
# Réduire la taille des données pour l'entraînement
df_small = df.sample(n=5000, random_state=42)
X_bow_small = X_bow[:5000]

# Répartition des données
X_train, X_test, y_train, y_test = train_test_split(X_bow_small, df_small['tags'], test_size=0.3, random_state=42)

#### Modelisation

In [35]:
# Modelisation avec Regression Logistique et GridSearchCV

# Initialisation de GridSearchCV et definir les hyperparametres
param_grid = {
    'C': [0.1, 1, 10, 100],  # Inverse de la force de régularisation
    'solver': ['lbfgs', 'liblinear', 'saga'],  # Choix du solveur
    'max_iter': [100, 200, 300]  # Nombre d'itérations
}

# Initialiser le classifieur
model_reg_log = LogisticRegression(solver='saga', max_iter=200)

# Initialiser GridSearchCV
grid_search = GridSearchCV(estimator=model_reg_log, param_grid=param_grid, cv=2, scoring='accuracy', n_jobs=-1)

# Entrainer GridSearchCV
model_reg_log.fit(X_train, y_train)

# Faire des predictions sur l'ensemble de test
y_pred = model_reg_log.predict(X_test)

#### Resultat et Scoring

#### Predire une phrase

In [ ]:
phrase = "Comment utiliser une boucle FOR en Python ?"

predicted_tags = model_reg_log.predict(phrase)

# Afficher les prédictions
print(f"Tags prédits pour la phrase '{phrase}' : {predicted_tags}")

ValueError: Expected 2D array, got scalar array instead:
array=Comment utiliser une boucle FOR en Python ?.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.